In [1]:
import sys
import os
import shutil
import datetime
import re
import pandas as pd
from collections import Counter
#sys.path.append(os.path.abspath("/home/jose/Dropbox/Doktorarbeit/"))
#from reading_robot import load_data, tei2text, text2features, classify, process_labels, describe_data,feature_analysis, cull_data, st_tests, metadata2numbers, cluster, call_stylo, prepare_subcorpus
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline
from shutil import copyfile
import numpy as np
import glob

# Read Data

In [2]:
cuentos_metadata_df = pd.read_csv("./../data/cuentos_espanoles/metadata.csv", index_col=0)
cuentos_metadata_df

,idno,author.name,title,year,supergenre,genre,subgenre.cligs.important,author.country,author.viaf,author.bne,...,am.verses,am.sps,am.pds,am.numbers,am.puncts,am.blocks,am.chapters,am.shortStories,am.fts,decade
ce0005,ce0005,Bazan,CuAmor,1898,narrative,colección-cuentos,NaN,Spain,51699645,NaN,...,4,0,259,68,12934,848,0,43,0,1890
ce0007,ce0007,Bazan,CuTierra,1922,narrative,colección-cuentos,NaN,Spain,51699645,NaN,...,6,0,470,67,14391,1190,0,43,0,1920
ce0009,ce0009,Bazan,CuMarineda,1892,narrative,colección-cuentos,NaN,Spain,51699645,NaN,...,0,0,183,23,8480,501,0,9,0,1890
ce0017,ce0017,Bazan,interiores,1907,narrative,colección-cuentos,NaN,Spain,51699645,NaN,...,0,75,186,53,7669,673,0,21,0,1900
ce0018,ce0018,BlascoIbanez,Luna_c,1909,narrative,colección-cuentos,NaN,Spain,73848096,NaN,...,0,0,53,0,4030,363,0,11,0,1900
ce0019,ce0019,BlascoIbanez,Flor,1895,narrative,colección-cuentos,realist,Spain,73848096,NaN,...,4,0,39,0,2227,372,7,4,0,1890
ce0020,ce0020,Clarin,Pipa,1886,narrative,colección-cuentos,NaN,Spain,17218138,NaN,...,42,0,636,22,15907,1537,25,9,0,1880
ce0021,ce0021,Clarin,Senor,1893,narrative,colección-cuentos,NaN,Spain,17218138,NaN,...,1,0,330,14,10847,859,18,13,1,1890
ce0022,ce0022,Clarin,Morales,1896,narrative,colección-cuentos,NaN,Spain,17218138,NaN,...,49,28,240,60,17884,1390,22,28,0,1890
ce0023,ce0023,Clarin,Berta,1892,narrative,colección-cuentos,NaN,Spain,17218138,NaN,...,15,0,181,7,9671,544,32,3,0,1890


In [3]:
cuentos_metadata_df['am.tokens']

ce0005    62451
ce0007    61119
ce0009    39980
ce0017    29341
ce0018    23761
ce0019    15269
ce0020    73806
ce0021    50435
ce0022    86709
ce0023    46266
ce0025    39520
ce0026    26167
ce0027    15228
ce0028    20728
ce0029    64795
ce0030    37398
ce0031    40020
ce0033    27525
ce0034    31960
ce0035    18438
Name: am.tokens, dtype: int64

In [4]:
cuentos_metadata_df["author.name"].value_counts()

Bazan           4
Clarin          4
Miro            4
Valle           3
BlascoIbanez    2
Unamuno         1
Pereda          1
Galdos          1
Name: author.name, dtype: int64

In [5]:
number_texts = cuentos_metadata_df.shape[0]
number_authors = len(cuentos_metadata_df["author.name"].unique())
mean_number_text_author = cuentos_metadata_df["author.name"].value_counts().mean()
std_number_text_author = cuentos_metadata_df["author.name"].value_counts().std()
mean_length_text = cuentos_metadata_df['am.tokens'].mean()
std_length_text = cuentos_metadata_df['am.tokens'].std()
distribution_authors_texts = cuentos_metadata_df["author.name"].value_counts()


In [6]:
subfolder = "teatro-sgo"

In [7]:
#P:\evaluation-of-parameters-private\data\teatro-sgo\only-tokens\directorio_base
wdir = "../data/" + subfolder + "/only-tokens/directorio_base/"

In [8]:
files = glob.glob(wdir + "/*.txt")


In [9]:
files

['../data/teatro-sgo/only-tokens/directorio_base\\Calderon_CasaConDosPuertasMalaEsDeGuardar.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Calderon_ElGranTeatroDelMundo.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Calderon_ElMedicoDeSuHonra.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Calderon_LaDamaDuende.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Calderon_LaVidaEsSueno.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Calderon_NoHayBurlasConElAmor.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Castro_ElCondeAlarcos.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Castro_LasMocedadesDelCidParteI.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Castro_LasMocedadesDelCidParteII.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Lope_ElCaballeroDeOlmedo.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Lope_ElPerroDelHortelano.txt',
 '../data/teatro-sgo/only-tokens/directorio_base\\Lope_ElVillanoEnSuRinco

In [10]:
len(files)

38

In [11]:
files_df = pd.DataFrame(files, columns=["path"])

In [12]:
files_df

,path
0,../data/teatro-sgo/only-tokens/directorio_base...
1,../data/teatro-sgo/only-tokens/directorio_base...
2,../data/teatro-sgo/only-tokens/directorio_base...
3,../data/teatro-sgo/only-tokens/directorio_base...
4,../data/teatro-sgo/only-tokens/directorio_base...
5,../data/teatro-sgo/only-tokens/directorio_base...
6,../data/teatro-sgo/only-tokens/directorio_base...
7,../data/teatro-sgo/only-tokens/directorio_base...
8,../data/teatro-sgo/only-tokens/directorio_base...
9,../data/teatro-sgo/only-tokens/directorio_base...


In [13]:
files_df["file_name"] = files_df["path"].str.findall("\\\(.*?)\.txt").str[0]

In [14]:
files_df

,path,file_name
0,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_CasaConDosPuertasMalaEsDeGuardar
1,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElGranTeatroDelMundo
2,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElMedicoDeSuHonra
3,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaDamaDuende
4,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaVidaEsSueno
5,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_NoHayBurlasConElAmor
6,../data/teatro-sgo/only-tokens/directorio_base...,Castro_ElCondeAlarcos
7,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteI
8,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteII
9,../data/teatro-sgo/only-tokens/directorio_base...,Lope_ElCaballeroDeOlmedo


In [15]:
files_df["author"] = files_df["file_name"].str.findall("^(.*?)_").str[0]

In [16]:
files_df["text_title"] = files_df["file_name"].str.findall("_(.*?)$").str[0]

In [17]:
files_df

,path,file_name,author,text_title
0,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_CasaConDosPuertasMalaEsDeGuardar,Calderon,CasaConDosPuertasMalaEsDeGuardar
1,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElGranTeatroDelMundo,Calderon,ElGranTeatroDelMundo
2,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElMedicoDeSuHonra,Calderon,ElMedicoDeSuHonra
3,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaDamaDuende,Calderon,LaDamaDuende
4,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaVidaEsSueno,Calderon,LaVidaEsSueno
5,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_NoHayBurlasConElAmor,Calderon,NoHayBurlasConElAmor
6,../data/teatro-sgo/only-tokens/directorio_base...,Castro_ElCondeAlarcos,Castro,ElCondeAlarcos
7,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteI,Castro,LasMocedadesDelCidParteI
8,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteII,Castro,LasMocedadesDelCidParteII
9,../data/teatro-sgo/only-tokens/directorio_base...,Lope_ElCaballeroDeOlmedo,Lope,ElCaballeroDeOlmedo


In [18]:
sorted(list(files_df.author.unique()))


['Calderon',
 'Castro',
 'Lope',
 'Moreto',
 'RojasZorrilla',
 'SJ',
 'Salazar',
 'Solis',
 'Tirso',
 'VélezGuevara']

In [19]:
for index, row in files_df.iterrows():
    with open(row["path"], "r", errors="replace", encoding="utf-8") as fin:
        content_str = fin.read()
    files_df.loc[index, "text"] = content_str

In [20]:
files_df

,path,file_name,author,text_title,text
0,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_CasaConDosPuertasMalaEsDeGuardar,Calderon,CasaConDosPuertasMalaEsDeGuardar,\nDon_Félix\nbésoos\nlas\nmanos\nEl\ncielo\nLi...
1,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElGranTeatroDelMundo,Calderon,ElGranTeatroDelMundo,\nYa\nestamos\na\ntu\nobediencia\nautor\nnuest...
2,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElMedicoDeSuHonra,Calderon,ElMedicoDeSuHonra,\nJesús\nmil\nveces\nEl\ncielo\nte\nvalga\nQué...
3,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaDamaDuende,Calderon,LaDamaDuende,\nPor\nun\nhora\nno\nllegamos\na\ntiempo\nde\n...
4,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaVidaEsSueno,Calderon,LaVidaEsSueno,\nDi\ndos\ny\nno\nme\ndejes\nen\nla\nposada\na...
5,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_NoHayBurlasConElAmor,Calderon,NoHayBurlasConElAmor,\nValga\nte\nel\ndiablo\nQué\ntienes\nque\nand...
6,../data/teatro-sgo/only-tokens/directorio_base...,Castro_ElCondeAlarcos,Castro,ElCondeAlarcos,\nVuelve\na\nmi\ncuello\nesos\nlazos\nde\nel\n...
7,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteI,Castro,LasMocedadesDelCidParteI,\nEs\ngran\npremio\na\nmi\nlealtad\nA\nlo\nque...
8,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteII,Castro,LasMocedadesDelCidParteII,\nSantiago\nSantiago\ncierra\nEspaña\ncierra\n...
9,../data/teatro-sgo/only-tokens/directorio_base...,Lope_ElCaballeroDeOlmedo,Lope,ElCaballeroDeOlmedo,\nAmor\nno\nte\nllame\namor\nel\nque\nno\nte\n...


In [21]:
files_df.index = files_df.file_name

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
reg_ex = r"(?u)\b[\w_@]+\b|[¶\(»\]\?\.\–\!’•\|“\>\)\-\—\:\}\*\&…¿\/=¡\"\'·+\{\#\[;­,«~]"
sort_columns_by_frequency = True
lowercase = True
max_features = 1000000
analyzer = "word" #{‘word’, ‘char’, ‘char_wb’}
ngram_range = (1,1) #ngram_range = None

#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(corpus)

#vectorizer = CountVectorizer()

vec = CountVectorizer(
        token_pattern = reg_ex,
        lowercase = lowercase,
        max_features = max_features,
        analyzer = analyzer,
        ngram_range = ngram_range,
        )

tokens = vec.fit_transform(files_df.text.tolist())

tokens_df = pd.DataFrame(tokens.toarray(), columns = vec.get_feature_names(), index = files_df["file_name"])

In [24]:
tokens_df

,-,.,1,12_de_marzo_de_1400,2,3,4,638,_baja,a,...,órdenes,última,últimas,último,últimos,única,único,úsanla,útil,реna
file_name,,,,,,,,,,,,,,,,,,,,,
Calderon_CasaConDosPuertasMalaEsDeGuardar,1,0,0,0,0,0,0,0,0,418,...,0,0,0,0,0,0,0,0,0,0
Calderon_ElGranTeatroDelMundo,0,0,0,0,0,0,0,0,0,179,...,0,0,0,0,0,0,0,0,0,0
Calderon_ElMedicoDeSuHonra,0,0,0,0,0,0,0,0,0,406,...,0,0,0,2,0,0,0,0,0,0
Calderon_LaDamaDuende,1,0,0,0,0,0,0,0,0,377,...,0,0,0,2,1,0,0,0,0,0
Calderon_LaVidaEsSueno,0,13,6,0,6,0,0,0,0,483,...,0,1,0,1,0,0,0,0,0,0
Calderon_NoHayBurlasConElAmor,0,0,0,0,0,0,0,0,0,408,...,0,0,0,0,0,0,0,0,0,0
Castro_ElCondeAlarcos,0,1,0,0,0,0,0,0,0,304,...,0,0,0,0,0,0,0,0,0,0
Castro_LasMocedadesDelCidParteI,0,0,0,0,0,0,0,0,0,382,...,0,0,0,0,0,0,0,0,0,0
Castro_LasMocedadesDelCidParteII,0,1,0,0,0,0,0,0,1,267,...,0,0,0,0,0,1,0,0,0,0


In [25]:
files_df["tokens_sum"] = tokens_df.sum(axis=1)

In [26]:
files_df

,path,file_name,author,text_title,text,tokens_sum
file_name,,,,,,
Calderon_CasaConDosPuertasMalaEsDeGuardar,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_CasaConDosPuertasMalaEsDeGuardar,Calderon,CasaConDosPuertasMalaEsDeGuardar,\nDon_Félix\nbésoos\nlas\nmanos\nEl\ncielo\nLi...,14214
Calderon_ElGranTeatroDelMundo,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElGranTeatroDelMundo,Calderon,ElGranTeatroDelMundo,\nYa\nestamos\na\ntu\nobediencia\nautor\nnuest...,6506
Calderon_ElMedicoDeSuHonra,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_ElMedicoDeSuHonra,Calderon,ElMedicoDeSuHonra,\nJesús\nmil\nveces\nEl\ncielo\nte\nvalga\nQué...,14999
Calderon_LaDamaDuende,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaDamaDuende,Calderon,LaDamaDuende,\nPor\nun\nhora\nno\nllegamos\na\ntiempo\nde\n...,15500
Calderon_LaVidaEsSueno,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_LaVidaEsSueno,Calderon,LaVidaEsSueno,\nDi\ndos\ny\nno\nme\ndejes\nen\nla\nposada\na...,16745
Calderon_NoHayBurlasConElAmor,../data/teatro-sgo/only-tokens/directorio_base...,Calderon_NoHayBurlasConElAmor,Calderon,NoHayBurlasConElAmor,\nValga\nte\nel\ndiablo\nQué\ntienes\nque\nand...,14271
Castro_ElCondeAlarcos,../data/teatro-sgo/only-tokens/directorio_base...,Castro_ElCondeAlarcos,Castro,ElCondeAlarcos,\nVuelve\na\nmi\ncuello\nesos\nlazos\nde\nel\n...,12834
Castro_LasMocedadesDelCidParteI,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteI,Castro,LasMocedadesDelCidParteI,\nEs\ngran\npremio\na\nmi\nlealtad\nA\nlo\nque...,14634
Castro_LasMocedadesDelCidParteII,../data/teatro-sgo/only-tokens/directorio_base...,Castro_LasMocedadesDelCidParteII,Castro,LasMocedadesDelCidParteII,\nSantiago\nSantiago\ncierra\nEspaña\ncierra\n...,8712


In [27]:
files_df.author.value_counts().mean()

3.8

In [28]:
files_df.shape[0]

38

In [29]:
files_df.author.value_counts()

Lope             6
Calderon         6
Tirso            5
SJ               5
RojasZorrilla    4
Moreto           3
Castro           3
VélezGuevara    2
Salazar          2
Solis            2
Name: author, dtype: int64

In [30]:
files_df.to_csv("./../data/"+ subfolder+ "/metadata.tsv", sep="\t")

In [31]:
number_texts = files_df.shape[0]
number_authors = len(files_df.author.unique())
mean_number_text_author = files_df.author.value_counts().mean()
std_number_text_author = files_df.author.value_counts().std()
mean_length_text = files_df.tokens_sum.mean()
std_length_text = files_df.tokens_sum.std()
distribution_authors_texts = files_df.author.value_counts()


In [32]:
summary_corpus_df = pd.DataFrame(
            [number_texts,   number_authors,  mean_number_text_author,   std_number_text_author,  mean_length_text, std_length_text, distribution_authors_texts  ],
    index=["number_texts","number_authors","mean_number_text_author", "std_number_text_author","mean_length","std_length", "distribution_authors_texts"  ],
    columns = ["16-17-teatro"]).T

In [33]:
summary_corpus_df

,number_texts,number_authors,mean_number_text_author,std_number_text_author,mean_length,std_length,distribution_authors_texts
16-17-teatro,38,10,3.8,1.61933,13011.6,4041.93,Lope 6 Calderon 6 Tirso ...


In [34]:
summary_corpus_df.to_csv("./../data/"+ subfolder+ "/summary.tsv", sep="\t")